# Learning Outside The Lines: Project your Data onto a Map

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from pyproj import Proj, transform

import folium

from json import dumps, load

#SOURCE OF DATA http://www.indy.gov/eGov/City/DPS/IMPD/Crimes/Pages/UCRDownload.aspx



### Understanding the Coordinate System 

The coordinate system for the IPS incident data spreadhseet is based on this state plane: 
NAD_1983_StatePlane_Indiana_East_FIPS_1301_Feet

I refered to this website to verify the accuracy of the data:
http://www.earthpoint.us/StatePlane.aspx
Select 1301 Indiana East, using the US Survey in Feet

ALSO READ THIS TO UNDERSTAND HOW TO HANDLE COORDINATE DATA IN ORDER TO PROJECT THEM ONTO A MAP:
http://downloads2.esri.com/support/documentation/ao_/710Understanding_Map_Projections.pdf

---------------------------------------------------------------------------------------------------------

In [2]:
inProj = Proj(init='esri:102673', preserve_units=True)
outProj = Proj(init='epsg:4326')

def get_long_lat(x):
    y_coord, x_coord = transform(inProj, outProj, x[0], x[1], radians=False)
    return x_coord, y_coord

In [3]:
columnspecs = [(0,5), (5, 33), (33, 44), (44, 51), (51, 63), (63, 108), (108, 115), (115, 137), (137,159)]

i2014 = pd.read_fwf("2014_incidents.txt", 
                    skiprows=1, 
                    names=['UCR','CRIME','DATE','TIME','CASE','ADDRESS','BEAT','X_COORD','Y_COORD'],
                    colspecs=columnspecs)

i2015 = pd.read_fwf("2015_incidents.txt", 
                    skiprows=1, 
                    names=['UCR','CRIME','DATE','TIME','CASE','ADDRESS','BEAT','X_COORD','Y_COORD'],
                    colspecs=columnspecs)

In [4]:
i2016 = pd.read_csv("2016_incidents.csv")
i2016 = i2016[i2016.columns.drop(['ID', 'OBJECTID'])]
i2016.columns = i2015.columns

In [5]:
all(i2014.dtypes == i2015.dtypes) == all(i2015.dtypes == i2016.dtypes)

True

In [6]:
df = i2014.append(i2015).copy()
df = df.append(i2016).copy()
df['GPS_COORD_X'], df['GPS_COORD_Y'] = zip(*df[['X_COORD', 'Y_COORD']].apply(get_long_lat, axis=1))

#Erase
i2014 = None
i2015 = None
i2016 = None

In [7]:
#NORTHEAST BOUNDARY
NE_B = (39.965782, -86.014354)
#SOUTHWEST BOUNDARY
SW_B = (39.638192, -86.362483)

df = df[df['GPS_COORD_X'].between(SW_B[0], NE_B[0])]
df = df.reset_index(drop=True)
df = df[df['GPS_COORD_Y'].between(SW_B[1], NE_B[1])]
df = df.reset_index(drop=True)

Crediting GEOJSON and TOPOJSON files to http://mapshaper.org/

In [8]:
m = folium.Map(location=[df.GPS_COORD_X.mean(), df.GPS_COORD_Y.mean()], tiles='Stamen Toner', zoom_start=11)

folium.GeoJson(load(open("shapefiles/Indianapolis_Police_Zones/geo_police_zones.json")), name='geojson').add_to(m)

folium.TopoJson(open("shapefiles/Indianapolis_Police_Zones/topo_police_zones.json"),
                "objects.Indianapolis_Police_Zones",
                name='topojson',
               ).add_to(m)

folium.LayerControl().add_to(m)

In [9]:
beat_incident_count = df[['BEAT', 'CASE']].groupby(by='BEAT', as_index=False).agg(np.count_nonzero)
beat_incident_count.columns = ['BEAT', 'CASE_COUNT']

In [10]:
def getmap(beat_map_breakdown):
    m = folium.Map(location=[df.GPS_COORD_X.mean(), df.GPS_COORD_Y.mean()], tiles='Stamen Toner', zoom_start=11)
    m.choropleth(
        load(open("shapefiles/Indianapolis_Police_Zones/geo_police_zones.json")),
        data=beat_incident_count,
        columns=['BEAT', 'CASE_COUNT'],
        key_on='properties.POLICEZONE',
        fill_color='YlOrRd',
        fill_opacity=0.3,
        highlight=True,
        threshold_scale=list(np.linspace(beat_map_breakdown['CASE_COUNT'].min(), 
                                         beat_map_breakdown['CASE_COUNT'].max(), 
                                         num=4))
        )

    gdf = gpd.read_file("shapefiles/Indianapolis_Police_Zones/geo_police_zones.json")
    gdf['X'] = gdf['geometry'].apply(lambda x: x.centroid.coords.xy[1][0])
    gdf['Y'] = gdf['geometry'].apply(lambda x: x.centroid.coords.xy[0][0])
    idx_chg = gdf[gdf['POLICEZONE']=='Excluded']['POLICEZONE'].index
    gdf.loc[idx_chg, 'POLICEZONE'] = gdf.loc[idx_chg]['JURISDCTN'] + "_" + gdf.loc[idx_chg]['OBJECTID'].astype(str)

    x = gdf.iterrows()
    print(x)
    while True:
        try:
            c = next(x)
            folium.Marker(location=[c[1]['X'], c[1]['Y']], popup=c[1]['POLICEZONE']).add_to(m)
        except:
            break
    return m

In [13]:
beat_incident_count = df[['BEAT', 'CASE']].groupby(by='BEAT', as_index=False).agg(np.count_nonzero)
beat_incident_count.columns = ['BEAT', 'CASE_COUNT']

m = getmap(beat_incident_count)

<generator object DataFrame.iterrows at 0x1100800a0>


In [14]:
m.save("awesome.html")